In [2]:
import cv2
import os
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np

import keras
from keras.layers import BatchNormalization, Dropout
from keras.layers import Conv3D, MaxPool3D, Conv2D, MaxPool2D, Flatten
from keras.layers import Conv2DTranspose, UpSampling2D
from keras.layers import Dense, Activation, Reshape
from keras.models import Input, Model
from keras.models import Sequential
from keras.optimizers import Adam

% matplotlib inline

Using TensorFlow backend.


In [3]:
def add_images_from_files_to_list(root, files, list, data_cb=None):
    if files:
        def identity(v):
            return v

        if data_cb is None:
            data_cb = identity

        for index, file in enumerate(files):
            image = cv2.imread(os.path.join(root, file))

            if image is not None:
                list.append(data_cb(image))
    return list

In [5]:
images_train = []

In [5]:

for root, sub_directories, files in tqdm(os.walk("./danbooru-faces")):
    add_images_from_files_to_list(root, files, images_train)

0it [00:00, ?it/s]

2it [00:00,  2.63it/s]

3it [00:01,  2.10it/s]

4it [00:02,  1.89it/s]

5it [00:02,  1.91it/s]

...

11it [00:06,  1.76it/s]

12it [00:06,  1.75it/s]

13it [00:07,  1.72it/s]

14it [00:08,  1.68it/s]

KeyboardInterrupt: 

In [6]:
images_train = np.array(images_train)
# images_train = images_train.reshape((images_train.shape[0],
#                       images_train.shape[3],
#                       images_train.shape[1],
#                       images_train.shape[2]))

In [7]:
print(images_train.shape)

(14901, 96, 96, 3)


In [8]:
dropout = 0.4
depth = 256
dim = 30
input_dim = 128
batch_size = 128
epochs = 200

In [9]:
def keras_generator():
    g = Sequential()
    # In: 100
    # Out: dim x dim x depth
    g.add(Dense(dim * dim * depth, input_dim=input_dim))
    g.add(BatchNormalization(momentum=0.9))
    g.add(Activation('relu'))
    g.add(Reshape((dim, dim, depth)))
    g.add(Dropout(dropout))
    # In: dim x dim x depth
    # Out: 2*dim x 2*dim x depth/2
    g.add(UpSampling2D())
    g.add(Conv2DTranspose(int(depth / 2), 5, padding='same'))
    g.add(BatchNormalization(momentum=0.9))
    g.add(Activation('relu'))
    g.add(UpSampling2D())
    g.add(Conv2DTranspose(int(depth / 4), 5, padding='same'))
    g.add(BatchNormalization(momentum=0.9))
    g.add(Activation('relu'))
    # In: dim x dim x depth
    # Out: 4*dim x 4*dim x depth/4
    g.add(UpSampling2D())
    g.add(Conv2DTranspose(int(depth / 8), 5, padding='same'))
    g.add(BatchNormalization(momentum=0.9))
    g.add(Activation('relu'))
    g.add(UpSampling2D())
    g.add(Conv2DTranspose(int(depth / 16), 5, padding='same'))
    g.add(BatchNormalization(momentum=0.9))
    g.add(Activation('relu'))
    # Out: 480 x 480 x 1 grayscale image [0.0,1.0] per pix
    g.add(Conv2DTranspose(1, 5, padding='same'))
    g.add(Activation('relu'))
    
    g.add(Reshape((480 * 480,)))
    
    # Out: 96 x 96 x 1 grayscale image
    g.add(Dense(96 * 96))
    g.add(Reshape((96, 96)))
    g.add(Activation('tanh'))
    g.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
    g.summary()
    return g

In [10]:
def keras_discriminator():
    """
        Translation of tutorial into Keras.
    """
    inputs = Input(shape=images_train.shape[1:])
    layer = Conv2D(filters=32, kernel_size=(5, 5), activation="relu")(inputs)
    layer = MaxPool2D()(layer)
    layer = Conv2D(filters=64, kernel_size=(5, 5), activation="relu")(layer)
    layer = MaxPool2D()(layer)

    layer = Flatten()(layer)
    layer = Dense(units=1024, activation="relu")(layer)
    layer = Dense(units=128, activation="relu")(layer)
    outputs = Dense(units=1, activation="sigmoid")(layer)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss="binary_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    model.summary()

    return model

In [11]:
# Optimizer
adam = Adam(lr=0.0002, beta_1=0.5)

In [13]:
generator1 = keras_generator()
generator2 = keras_generator()
generator3 = keras_generator()

ganInput = Input((input_dim,))

x1 = generator1(ganInput)
x2 = generator2(ganInput)
x3 = generator3(ganInput)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 230400)            29721600  
_________________________________________________________________
batch_normalization_16 (Batc (None, 230400)            921600    
_________________________________________________________________
activation_22 (Activation)   (None, 230400)            0         
_________________________________________________________________
reshape_10 (Reshape)         (None, 30, 30, 256)       0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 30, 30, 256)       0         
_________________________________________________________________
up_sampling2d_13 (UpSampling (None, 60, 60, 256)       0         
_________________________________________________________________
conv2d_transpose_16 (Conv2DT (None, 60, 60, 128)       819328    
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 230400)            29721600  
_________________________________________________________________
batch_normalization_21 (Batc (None, 230400)            921600    
_________________________________________________________________
activation_29 (Activation)   (None, 230400)            0         
_________________________________________________________________
reshape_13 (Reshape)         (None, 30, 30, 256)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 30, 30, 256)       0         
_________________________________________________________________
up_sampling2d_17 (UpSampling (None, 60, 60, 256)       0         
_________________________________________________________________
conv2d_transpose_21 (Conv2DT (None, 60, 60, 128)       819328    
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 230400)            29721600  
_________________________________________________________________
batch_normalization_26 (Batc (None, 230400)            921600    
_________________________________________________________________
activation_36 (Activation)   (None, 230400)            0         
_________________________________________________________________
reshape_16 (Reshape)         (None, 30, 30, 256)       0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 30, 30, 256)       0         
_________________________________________________________________
up_sampling2d_21 (UpSampling (None, 60, 60, 256)       0         
_________________________________________________________________
conv2d_transpose_26 (Conv2DT (None, 60, 60, 128)       819328    
__________

In [15]:
discriminator = keras_discriminator()
d_input = keras.layers.concatenate([x1, x2, x3], axis=-1)
d_input = Reshape((96, 96, 3))(d_input)
generator = Model(inputs=ganInput, outputs=d_input)
ganOutput = discriminator(d_input)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 92, 92, 32)        2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 46, 46, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 42, 42, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 21, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28224)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              28902400  
__________

In [16]:
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [17]:
dLosses = []
gLosses = []

In [18]:
# Plot the loss from each batch
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('./collected_data/gan_loss_epoch_%d.png' % epoch)
    plt.clf()

In [19]:
# Create a wall of generated MNIST images
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, input_dim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('./collected_data/gan_generated_image_epoch_%d.png' % epoch)
    plt.clf()

In [20]:
# Save the generator and discriminator networks (and weights) for later use
def saveModels(epoch):
    generator1.save('./collected_models/gan_generator1_epoch_%d.h5' % epoch)
    generator2.save('./collected_models/gan_generator2_epoch_%d.h5' % epoch)
    generator3.save('./collected_models/gan_generator3_epoch_%d.h5' % epoch)
    discriminator.save('./collected_models/gan_discriminator_epoch_%d.h5' % epoch)

In [21]:
def train(epochs=1, batchSize=128):
    batchCount = int(images_train.shape[0] / batchSize)
    print('Epochs:', epochs)
    print('Batch size:', batchSize)
    print('Batches per epoch:', batchCount)

    for e in range(1, epochs+1):
        print('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in range(batchCount):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batchSize, input_dim])
            imageBatch = images_train[np.random.randint(0, images_train.shape[0], size=batchSize)]

            # Generate fake MNIST images
            generatedImages = generator.predict(noise)
            # print np.shape(imageBatch), np.shape(generatedImages)
            X = np.concatenate([imageBatch, generatedImages])

            # Labels for generated and real data
            yDis = np.zeros(2*batchSize)
            # One-sided label smoothing
            yDis[:batchSize] = 0.9

            # Train discriminator
            discriminator.trainable = True
            dloss = discriminator.train_on_batch(X, yDis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batchSize, input_dim])
            yGen = np.ones(batchSize)
            discriminator.trainable = False
            gloss = gan.train_on_batch(noise, yGen)

        # Store loss of most recent batch from this epoch
        dLosses.append(dloss)
        gLosses.append(gloss)

        if e == 1 or e % 20 == 0:
            plotGeneratedImages(e)
            saveModels(e)

    # Plot losses from every epoch
    plotLoss(e)

In [22]:
train(epochs, batch_size)

Epochs: 200
Batch size: 128
Batches per epoch: 116
--------------- Epoch 1 ---------------
